# RSA analysis of the Neuropixel Data

This notebook analyzes Allen Institute's Neuropixel data. It focuses on the `functional_connectivity` dataset since that dataset seems to be richer (more repeats of the same stimuli), with more rest time in-between sessions, and has better quality data (only one animal missing pupilometry measurements).

In [1]:
from src.neuropixel import Neuropixel

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
HOME = Path('/media/internal/projects/Allen/Ecephys')
npx = Neuropixel(HOME)

In [3]:
data_type = 'functional_connectivity'

## Natural Movie 1

This movie is played in both sessions of `functional_connectivity`, each repeated 60 times on each session (there are also 20 repeats with temporal shuffle that I'm going to ignore for now). The movie has 30s duration, so I will extract 30 "blocks" of 1s duration.

Let's start by getting the dataframes for each of these blocks.

In [4]:
table = npx.cache.get_session_table()
session_ids = table.loc[table.session_type == data_type].index.values

In [5]:
# remove sessions w/o eye data
session_ids = np.delete(session_ids, np.where(session_ids == 768515987))
session_ids = np.delete(session_ids, np.where(session_ids == 840012044))

In [6]:
session_ids

array([766640955, 767871931, 771160300, 771990200, 774875821, 778240327,
       778998620, 779839471, 781842082, 786091066, 787025148, 789848216,
       793224716, 794812542, 816200189, 819186360, 819701982, 821695405,
       829720705, 831882777, 835479236, 839068429, 839557629, 847657808])

In [7]:
movie_name = "natural_movie_one_more_repeats"
rois = ["VISrl", "VISp", "VISpm", "VISam", "VISal", "VISl"]
# rois = ["VISp"]
seconds = np.arange(0, 30)

In [8]:
dfs = []
for s, sess in enumerate(session_ids):
 print(f"Session {s + 1 } of {len(session_ids)}:")
 for sec in seconds:
  print(f"Second {sec + 1} of 30.")
  for r, roi in enumerate(rois):

   print(f"ROI {r + 1} of {len(rois)}.")

   # load the session data
   npx.session = sess

   # get the spike counts
   df = npx.get_movie_spike_count(movie_name, roi, sec)

   dfs.append(df)
 print('-----------------------------------------')

Session 1 of 24:
Second 1 of 30.
ROI 1 of 6.
ROI 2 of 6.
ROI 3 of 6.


/home/devxl/miniconda3/envs/neuro/lib/python3.7/site-packages/allensdk/brain_observatory/ecephys/ecephys_session.py:1311: UserWarning: filtering to an empty set of units!
  warnings.warn(f'filtering to an empty set of {key}!')


KeyError: 'unit_id'

In [1]:
s = npx.cache.get_session_data(768515987)

NameError: name 'npx' is not defined

In [7]:
s.metadata

{'specimen_name': 'C57BL/6J-412809',
 'session_type': 'functional_connectivity',
 'full_genotype': 'wt/wt',
 'sex': 'M',
 'age_in_days': 136.0,
 'rig_equipment_name': 'NP.1',
 'num_units': 802,
 'num_channels': 2217,
 'num_probes': 6,
 'num_stimulus_presentations': 77361,
 'session_start_time': datetime.datetime(2018, 11, 5, 12, 25, 12, tzinfo=tzoffset(None, -28800)),
 'ecephys_session_id': 768515987,
 'structure_acronyms': [nan,
  'MGm',
  'MGv',
  'MGd',
  'TH',
  'LGd',
  'CA3',
  'DG',
  'CA1',
  'VISal',
  'MB',
  'APN',
  'NOT',
  'VISam',
  'Eth',
  'LP',
  'POL',
  'VISmma',
  'ProS',
  'VISp',
  'CA2',
  'VISl',
  'RPF',
  'VISrl'],
 'stimulus_names': ['spontaneous',
  'gabors',
  'flashes',
  'drifting_gratings_contrast',
  'natural_movie_one_more_repeats',
  'natural_movie_one_shuffled',
  'drifting_gratings_75_repeats',
  'dot_motion']}

In [9]:
a = np.array([
])

In [11]:
a.size

0